In [ ]:
import numpy as np
import pickle
from pinocchio.libpinocchio_pywrap import SE3
from locomote import  ContactSequenceHumanoid
from database import *
from croc_lib import *
from locomote.liblocomote_pywrap import ContactPatch
import time
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

#from utils import *
np.set_printoptions(precision=4,suppress=True)
from IPython.display import clear_output
from regression import *

### Visualize in ROS

In [ ]:
viz_talos = Visual()

### Create default problem

In [ ]:
croc_timestep = 0.0375
timesteps = np.array([8,32,8])

In [ ]:
init_state = np.array([-1.0006, -0.1115,  1.0231,  0.0001, -0.0004,  0.    ,  1.    ,
       -0.    ,  0.002 , -0.4209,  0.8599, -0.4382, -0.0039, -0.    ,
        0.002 , -0.421 ,  0.86  , -0.4383, -0.0039,  0.    ,  0.0067,
        0.2599,  0.1731, -0.0002, -0.5245,  0.    ,  0.    ,  0.1001,
        0.    , -0.2599, -0.174 , -0.0002, -0.5246,  0.    , -0.0001,
        0.1001,  0.    , -0.    ,  0.    ])

In [ ]:
#initialisation with zeros, except the initial state
trajs = []
for i in timesteps:
    traj = list(np.zeros([i,39]))
    trajs.append(traj)

trajs[0][0] = init_state

vel_trajs = []
for i in timesteps:
    traj = list(np.zeros([i,38]))
    vel_trajs.append(traj)


In [ ]:
#define contacts
contact_model_placement = SE3()

contact_model_placement.rotation = np.eye(3)

contact_model_placement.translation = np.array([0.,0.,-0.107])[:,None]

lfs2 = []

#define the first left foot location
lf_placement = SE3()
lf_placement.rotation = np.eye(3)
lf_placement.translation = np.array([-1, -0.02, 0.111])[:,None]
lf = ContactPatch(lf_placement)
lf.contactModelPlacement.translation = np.array([0.,0.,-0.107])[:,None]
lf.active = True
lfs2.append(lf)

#define the second left foot location
lf_placement = SE3()
lf_placement.rotation = np.eye(3)
lf_placement.translation = np.array([-1, -0.02, 0.111])[:,None]
lf = ContactPatch(lf_placement)
lf.contactModelPlacement.translation = np.array([0.,0.,-0.107])[:,None]
lf.active = False
lfs2.append(lf)

#define the third left foot location
lf_placement = SE3()
lf_placement.rotation = np.eye(3)
lf_placement.translation = np.array([-0.86, 0.07, 0.111])[:,None]
lf = ContactPatch(lf_placement)
lf.contactModelPlacement.translation = np.array([0.,0.,-0.107])[:,None]
lf.active = True
lfs2.append(lf)

#define the first, second and third right foot location
rf_placement = SE3()
rf_placement.rotation = np.eye(3)
rf_placement.translation = np.array([-1, -0.19, 0.111])[:,None]
rf = ContactPatch(rf_placement)
rf.contactModelPlacement.translation = np.array([0.,0.,-0.107])[:,None]
rf.active = True
rfs2 = [rf,rf,rf]

### Store into the format of crocoddyl

In [ ]:
phases = store_for_crocoddyl(timesteps, lfs2, rfs2, trajs, vel_trajs, croc_timestep)

In [ ]:
ROBOT = loadTalos('/opt/openrobots/share/example-robot-data')

In [ ]:
angles = np.arange(0,np.pi,np.pi/100)
step_length = 0.2
vec = []
for angle in angles:
    vec.append(step_length*np.array([np.cos(angle),np.sin(angle)]))
    
vec =np.array(vec)
plt.plot(-vec[:,1],vec[:,0],'*')
plt.xlabel('y')
plt.ylabel('x')

plt.axis([-0.3,0.1,-0.22,0.22])
plt.axis('equal')
plt.savefig('angles.png')

In [ ]:
vec =np.array(vec)

### Run Crocoddyl 

In [ ]:
x_input = []
x_output = []
for displacement in vec:
    trans = np.array([-1, -0.02, 0.004])[:,None]    
    trans[0,0] += displacement[0]
    trans[1,0] += displacement[1]
    phases[1]['left_sole_link'].translation = trans
    print phases[1]['left_sole_link'] 

    pickle.dump(phases,open('data_teguh_3.txt','wb'))
    problem, xs, us, ts = define_croc_problem(ROBOT,'data_teguh_3.txt', is_warmstart=False)
    solver = solve_problem(ROBOT,problem,xs, us)
    #raw_input()
    traj = np.array(solver.xs)[:,0:39]
    viz_talos.vis_traj(traj)
    x_input.append(displacement)
    x_output.append(solver.xs)
    print 'Length of the data is now ' + str(len(x_input))
    

x_input = np.array(x_input)
x_output = np.array(x_output)
data = dict()
data['input'] = x_input
data['output'] = x_output
f = open('data/one_step.pkl', 'wb')
pickle.dump(data, f)
f.close()

### Plot trajectories of joint angles 

In [ ]:
for i in range(traj.shape[1]):
    plt.plot(traj[:,i])
    plt.show()
    time.sleep(1)
    clear_output(wait=True)

### Load the data 

In [ ]:
viz_talos = Visual()

In [ ]:
f = open('data/one_step.pkl', 'rb')
data = pickle.load(f)
f.close()

In [ ]:
x_input = data['input'] 
x_output = data['output']
x_output_pose = x_output[:,:,0:7]
x_output_joint = x_output[:,:,7:39]
n = len(x_input)
timestep = x_output.shape[1]
print timestep

In [ ]:
trajs = x_output[:,:,0:39]

In [ ]:
for i in range(100):
    traj = trajs[np.random.randint(100),:,:]
    viz_talos.vis_traj(traj)
    time.sleep(0.5)

In [ ]:
x_output_pose = x_output_pose.reshape(n,-1)
x_output_joint = x_output_joint.reshape(n,-1)

In [ ]:
from sklearn.decomposition import PCA
joint_pca = PCA(n_components = 4)
pose_pca = PCA(n_components = 4)

joint_pca.fit(x_output_joint)
pose_pca.fit(x_output_pose)

print(joint_pca.explained_variance_)
print(joint_pca.singular_values_)

print(pose_pca.explained_variance_)
print(pose_pca.singular_values_)

joints_reduced = joint_pca.transform(x_output_joint)
poses_reduced = pose_pca.transform(x_output_pose)

In [ ]:
angles = np.arctan2(x_input[:,1], x_input[:,0])

In [ ]:
dpglm_joint = DP_GLM_Regressor(is_transform=True)

dpglm_joint.pca = joint_pca

dpglm_joint.fit(angles[:,None], joints_reduced)

In [ ]:
dpglm_pose = DP_GLM_Regressor(is_transform=True)

dpglm_pose.pca = pose_pca

dpglm_pose.fit(angles[:,None], poses_reduced)

In [ ]:
from geometry_msgs.msg import Quaternion, Pose, Point, Vector3
from std_msgs.msg import Header, ColorRGBA
from visualization_msgs.msg import Marker

In [ ]:
import rospy

In [ ]:
rospy.init_node('marker_publisher')

In [ ]:
marker_publisher = rospy.Publisher('visualization_marker', Marker, queue_size=5)
rospy.sleep(0.5)   
marker = Marker(
            type=Marker.CUBE,
            id=0,
            lifetime=rospy.Duration(5000),
            pose=Pose(Point(0., 0., 0.), Quaternion(0, 0, 0, 1)),
            scale=Vector3(0.25, 0.13, 0.01),
            header=Header(frame_id='world'),
            color=ColorRGBA(0.0, 1.0, 0.0, 0.8))
marker_publisher.publish(marker)

In [ ]:
joint_predict = []
pose_predict = []
for i in range(len(x_input)):
    angle = np.array([np.arctan2(x_input[i,1], x_input[i,0])])

    y,_= dpglm_joint.predict(angle[:,None],False)
    joint_predict.append(y[0,:])
    
    y,_= dpglm_pose.predict(angle[:,None],False)
    pose_predict.append(y[0,:])
    
joint_predict = np.array(joint_predict)
pose_predict = np.array(pose_predict)

In [ ]:
plt.plot(angles, poses_reduced,'r',linewidth=10)
plt.plot(angles, pose_predict,'b')

plt.show()

In [ ]:
ax = plt.axes()

In [ ]:
for i in range(4):
    plt.figure()
    plt.plot(angles, joints_reduced[:,i],'r',linewidth=3, label='true')
    plt.plot(angles, joint_predict[:,i],'b*',markersize=3, label='predicted')
    plt.xlabel(r'${\bf{\theta}}$',fontsize=16,fontweight='bold')
    plt.ylabel(r'$\bf{y}$'+str(i+1),fontsize=16)
    plt.legend(loc='best')
    ax = plt.axes()
    ax.tick_params(axis='both', which='major', labelsize=14)
    plt.tight_layout()
    plt.savefig('comp' + str(i)+'.png',dpi=500)

In [ ]:
plt.xticks('labelsize'='x-large')

In [ ]:
plt.subplot(2,2,1)
plt.plot(angles[:, joints_reduced,'r',linewidth=10)
plt.plot(angles, joint_predict,'b')

plt.subplot(2,2,2)
plt.plot(angles, joints_reduced,'r',linewidth=10)
plt.plot(angles, joint_predict,'b')

plt.subplot(2,2,3)
plt.plot(angles, joints_reduced,'r',linewidth=10)
plt.plot(angles, joint_predict,'b')

plt.subplot(2,2,4)
plt.plot(angles, joints_reduced,'r',linewidth=10)
plt.plot(angles, joint_predict,'b')


In [ ]:
step_length = 0.2
for i in range(1000):
    angle = np.random.rand()*np.pi
    #x = np.array(step_length*np.array([np.cos(angle),np.sin(angle)]))[None,:]
    x = np.array([angle])[None,:]
    #index = np.random.randint(len(x_input))
    #x = x_input[index][None,:]
    marker.pose.position.x = step_length*np.cos(angle)-1.
    marker.pose.position.y = step_length*np.sin(angle)
    marker_publisher.publish(marker)
    y_pose,_ = dpglm_pose.predict(x,False)
    print y_pose
    y_pose = dpglm_pose.pca.inverse_transform(y_pose)
    y_pose = y_pose.reshape(timestep,-1)
    
    y_joint,_ = dpglm_joint.predict(x,False)
    #print y_joint
    y_joint = dpglm_joint.pca.inverse_transform(y_joint)
    y_joint = y_joint.reshape(timestep, -1)
    y_predict = np.hstack([y_pose, y_joint])
    #print y_predict.shape

    traj = y_predict
    viz_talos.vis_traj(traj)